In [1]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence
import pandas as pd
import numpy as np
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-06-18 15:02:13.757927: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-18 15:02:14.686019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from mlp_encoder_model import *

In [3]:
model=CreateModel((16,16,2))
model.summary()

2025-06-18 15:02:16.397773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1209 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:01:00.0, compute capability: 8.0


Model: "smrtpxl_regression"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_pxls/ (InputLayer)    [(None, 16, 16, 2)]          0         []                            
                                                                                                  
 average_pooling2d (Average  (None, 16, 1, 2)             0         ['input_pxls/[0][0]']         
 Pooling2D)                                                                                       
                                                                                                  
 average_pooling2d_1 (Avera  (None, 1, 16, 2)             0         ['input_pxls/[0][0]']         
 gePooling2D)                                                                                     
                                                                                 

In [4]:
# get best weights file
pitch = '50x12P5'
batch_size = 5000
fingerprint = '34c2da80'
timeslices = 2
files = os.listdir('/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-mlp-sse-slim-checkpoints'.format(pitch, batch_size, fingerprint, timeslices))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
model.load_weights('/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-mlp-sse-slim-checkpoints/'.format(pitch, batch_size, fingerprint, timeslices)+bestfile)

print('Best model: {}'.format(bestfile))

Best model: weights.909-t30.28-v30.00.hdf5


In [5]:
# load in the test set
test_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = '/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val_slim',
    quantize = True
)

Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val_slim/metadata.json


In [6]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','y','cotB'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotBtrue'] = complete_truth[:,2]

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/dataset_3src_16x16/test-dataset_3src_16x16/{}_bs{}_{}_{}t_train-dataset_3src_16x16_test-dataset_3src_16x16_mlp-slim-sse-vars.parquet".format(pitch, batch_size, fingerprint, timeslices))

 1/21 [>.............................] - ETA: 10s

2025-06-18 15:02:30.669167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-06-18 15:02:30.674938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


21/21 [==============================] - 7s 327ms/step
